# **Overview**

This notebook demonstrates how to build a stateful, multi-step LLM workflow using LangGraph and LangChain.

**The workflow:**

Takes a topic as input

Uses an LLM to generate a joke

Uses the LLM again to explain the joke

Stores every step in memory

Allows time travel, state inspection, and state updates

**This notebook is designed to help understand:**

How LLM workflows are structured

How state flows between steps

How checkpoints and memory work in LangGraph

How LangGraph turns LLM calls into reliable systems

![Description](langgraph_joke_workflow.png)

In [64]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from langchain_openai import ChatOpenAI
# from langchain_core.messages import SystemMessage, HumanMessage
# import operator
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

Environment Variables

This notebook uses an OpenAI API key loaded from a .env file.

In [65]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

load_dotenv()
llm = ChatOpenAI(api_key=api_key)  # Replace with your OpenAI API key

This defines the shared memory (state) that flows through the workflow.

Field	        Purpose
topic       ->	Input provided by the user
joke        ->	Generated by the first node
explanation	->  Generated by the second node

Each node:

Reads from the state

Returns partial updates

LangGraph merges updates automatically

In [66]:
class JokeState(TypedDict):
    
    topic: str
    joke: str
    explanation: str

workflow nodes for generate_joke

Reads topic

Generates a joke using the LLM

Writes joke back into state

In [67]:
def generate_joke(state: JokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {"joke": response}

workflow nodes for generate_explanation

Reads joke

Generates an explanation

Writes explanation into state

In [68]:
def generate_explanation(state: JokeState):
    prompt = f'Write an explanation for the joke: {state["joke"]}'
    response = llm.invoke(prompt).content

    return {"explanation": response}

# Graph Structure
START → generate_joke → generate_explanation → END

This defines:

Execution order

Data flow

Completion point

In [69]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)


# Checkpointing & Memory

Automatic state snapshots

Workflow replay

Time travel

Debugging and inspection

⚠️ Memory is ephemeral (lost when kernel restarts)

In [70]:
checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

# Running the Workflow
Example 1: Topic = Pizza

Thread ID

Identifies an independent workflow session

Multiple threads can run in parallel

In [71]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({"topic":"pizza"}, config=config1)

{'topic': 'pizza',
 'joke': 'Why was the pizza chef bad at sword fighting? \nBecause he always tried to slice his opponents with pepperoni instead of a sword!',
 'explanation': 'This joke plays on the idea that a pizza chef would not be skilled in sword fighting because their expertise lies in making pizzas, not wielding weapons. The punchline adds to the humor by suggesting that the chef would mistakenly use pepperoni slices as a weapon instead of a traditional sword. This unexpected twist adds to the absurdity of the situation and helps make the joke funny.'}

# Inspecting State
Current State

Returns:

Current values

Next node

Checkpoint metadata

In [72]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza chef bad at sword fighting? \nBecause he always tried to slice his opponents with pepperoni instead of a sword!', 'explanation': 'This joke plays on the idea that a pizza chef would not be skilled in sword fighting because their expertise lies in making pizzas, not wielding weapons. The punchline adds to the humor by suggesting that the chef would mistakenly use pepperoni slices as a weapon instead of a traditional sword. This unexpected twist adds to the absurdity of the situation and helps make the joke funny.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1076-02be-6381-8002-4b43d6a973aa'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-14T05:10:42.903922+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1075-f0e7-6fe1-8001-13f2f695392a'}}, tasks=(), interrupts=())

# Full State History

Shows:

Every step

Input → joke → explanation

Parent-child relationships

This is extremely useful for:

Debugging

Auditing

Visualizing execution

In [73]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza chef bad at sword fighting? \nBecause he always tried to slice his opponents with pepperoni instead of a sword!', 'explanation': 'This joke plays on the idea that a pizza chef would not be skilled in sword fighting because their expertise lies in making pizzas, not wielding weapons. The punchline adds to the humor by suggesting that the chef would mistakenly use pepperoni slices as a weapon instead of a traditional sword. This unexpected twist adds to the absurdity of the situation and helps make the joke funny.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1076-02be-6381-8002-4b43d6a973aa'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-14T05:10:42.903922+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1075-f0e7-6fe1-8001-13f2f695392a'}}, tasks=(), interrupts=()),
 StateSnapshot(values

In [74]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({"topic":"pasta"}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti break up with the linguine? \n\nBecause they just couldn\'t "meatball" their relationship work!',
 'explanation': 'This joke plays on the fact that spaghetti and linguine are both types of pasta, so in the joke they are portrayed as a couple in a relationship. The punchline refers to the idea of "meatball," which sounds like "make all," suggesting that they just couldn\'t make their relationship work despite their best efforts. It\'s a light-hearted play on words that adds a humorous twist to the idea of pasta breaking up with each other.'}

In [75]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti break up with the linguine? \n\nBecause they just couldn\'t "meatball" their relationship work!', 'explanation': 'This joke plays on the fact that spaghetti and linguine are both types of pasta, so in the joke they are portrayed as a couple in a relationship. The punchline refers to the idea of "meatball," which sounds like "make all," suggesting that they just couldn\'t make their relationship work despite their best efforts. It\'s a light-hearted play on words that adds a humorous twist to the idea of pasta breaking up with each other.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1076-2275-6972-8002-77061e385b75'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-14T05:10:46.229616+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1076-1251-6917-8001-ecfdd47b207b'}}, tasks=(), interrup

# TIME TRAVEL

Why this matters:

Resume workflows from any point

Debug incorrect generations

Modify state and re-run

In [76]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0f06e4-04b1-6fbd-8002-a8d13ae3e95f"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0f06e4-04b1-6fbd-8002-a8d13ae3e95f'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [77]:
# workflow.invoke({}, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0f06e4-04b1-6fbd-8002-a8d13ae3e95f"}})
workflow.invoke(
    {"topic": "pizza"},
    {
        "configurable": {
            "thread_id": "1",
            "checkpoint_id": "1f0f06e4-04b1-6fbd-8002-a8d13ae3e95f"
        }
    }
)


{'topic': 'pizza',
 'joke': 'Why did the pizza go to the party? Because it wanted to get a pizza the action!',
 'explanation': 'This joke plays off the phrase "get a piece of the action," which means to be involved or participate in something exciting or interesting. In this case, the pizza literally wants to "get a pizza" (piece) of the action at the party, emphasizing the pun on the word "piece" as it relates to pizza. It\'s a light-hearted and playful way to imagine a pizza wanting to join in on the fun at a party.'}

In [78]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party? Because it wanted to get a pizza the action!', 'explanation': 'This joke plays off the phrase "get a piece of the action," which means to be involved or participate in something exciting or interesting. In this case, the pizza literally wants to "get a pizza" (piece) of the action at the party, emphasizing the pun on the word "piece" as it relates to pizza. It\'s a light-hearted and playful way to imagine a pizza wanting to join in on the fun at a party.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1076-3e04-6f04-8002-7cc7a685215a'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-14T05:10:49.119495+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1076-311a-66e4-8001-c4a018a8a553'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the p

# Updating State

What happens:

Overwrites part of the state

Creates a new branch in history

Future runs use updated values

In [79]:
workflow.update_state({"configurable": {"thread_id":"1", "checkpoint_id":"1f0f0b64-6df9-6d1f-8000-7e496f33741c", "checkpoint_ns":""}}, {"topic":"samosa"})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0f1076-3e70-69d7-8000-e0b65e4fbbf0'}}

In [80]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1076-3e70-69d7-8000-e0b65e4fbbf0'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2026-01-14T05:10:49.163595+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f0b64-6df9-6d1f-8000-7e496f33741c'}}, tasks=(PregelTask(id='c1a21f66-38d5-013d-dca5-ad7ae804b839', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the party? Because it wanted to get a pizza the action!', 'explanation': 'This joke plays off the phrase "get a piece of the action," which means to be involved or participate in something exciting or interesting. In this case, the pizza literally wants to "get a pizza" (piece) of the action at the party,

This produces a new joke & explanation based on updated state.

In [81]:
workflow.invoke(
    {"topic": "samosa"},
    {
        "configurable": {
            "thread_id": "1",
            "checkpoint_id": "1f0f0b7f-493c-6ced-8001-1477af215b28"
        }
    }
)

{'topic': 'samosa',
 'joke': "Why did the samosa break up with the onion bhaji?\n\nBecause it couldn't handle the tears!",
 'explanation': "This joke plays on the fact that both samosas and onion bhajis are popular Indian snacks that are often enjoyed together. The punchline refers to the fact that onions can make people cry when they are chopped, so the joke is implying that the samosa broke up with the onion bhaji because it couldn't handle the tears that came from the onions in the bhaji. It's a playful and light-hearted way to combine food and relationships in a humorous way."}

In [82]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': "Why did the samosa break up with the onion bhaji?\n\nBecause it couldn't handle the tears!", 'explanation': "This joke plays on the fact that both samosas and onion bhajis are popular Indian snacks that are often enjoyed together. The punchline refers to the fact that onions can make people cry when they are chopped, so the joke is implying that the samosa broke up with the onion bhaji because it couldn't handle the tears that came from the onions in the bhaji. It's a playful and light-hearted way to combine food and relationships in a humorous way."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1076-4db5-65a1-8002-c346e85f1b3f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-01-14T05:10:50.764617+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0f1076-446c-65b5-8001-571aff1324fe'}}, tasks=(), interrupts=()),
